# Deploy machine learning models to Azure

description: (preview) deploy your machine learning or deep learning model as a web service in the Azure cloud.

# Connect to your workspace

In [18]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='chrjia-eastus2euap2', subscription_id='13e50845-67bc-4ac5-94db-48d493a6d9e8', resource_group='chrjia-rg')

# Register your model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.



## Register a model from a local file

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [19]:
import urllib.request
from azureml.core.model import Model
# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", 'model.onnx')

# Register model
model = Model.register(ws, model_name='bidaf_onnx', model_path='./model.onnx')

Registering model bidaf_onnx


# Define an inference configuration

The inference configuration below specifies that the machine learning deployment will use the file echo_score.py in the ./source_dir directory to process incoming requests and that it will use the Docker image with the Python packages specified in the project_environment environment.

In [5]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='azmlenv')
inference_config = InferenceConfig(environment=env, source_directory='./src', entry_script='./echo_score.py')

## Define a deployment configuration

In [6]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

## Deploy your machine learning model

A deployment configuration specifies the amount of memory and cores to reserve for your webservice will require in order to run, as well as configuration details of the underlying webservice. For example, a deployment configuration lets you specify that your service needs 2 gigabytes of memory, 2 CPU cores, 1 GPU core, and that you want to enable autoscaling.

The options available for a deployment configuration differ depending on the compute target you choose. In a local deployment, all you can specify is which port your webservice will be served on.

In [7]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
2021/04/29 12:42:43 Downloading source code...
2021/04/29 12:42:44 Finished downloading source code
2021/04/29 12:42:44 Creating Docker network: acb_default_network, driver: 'bridge'
2021/04/29 12:42:45 Successfully set up Docker network: acb_default_network
2021/04/29 12:42:45 Setting up Docker configuration...
2021/04/29 12:42:45 Successfully set up Docker configuration
2021/04/29 12:42:45 Logging in to registry: chrjiaeastusc6ba5291.azurecr.io
2021/04/29 12:42:46 Successfully logged into chrjiaeastusc6ba5291.azurecr.io
2021/04/29 12:42:46 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/04/29 12:42:46 Scanning for dependencies...
2021/04/29 12:42:47 Successfully scanned dependencies
2021/04/29 12:42:47 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210301.v1@sha256:000d6c43f606

Removing intermediate container 1334818af10b
 ---> 8434c3ed2f11
Step 9/18 : ENV PATH /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/bin:$PATH
 ---> Running in 30fa2e7dd231
Removing intermediate container 30fa2e7dd231
 ---> 19a154bfb20d
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> 808df95c8eeb
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> a18efe51dff7
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad
 ---> Running in 5b20733526a9
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with materialized conda environment details
Successfully sent materialized environment details
Removing intermediate container 5b20733526a9
 ---> a2e1c73eebf2
Step 13/18 : ENV AZUREML

Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjEzZTUwODQ1LTY3YmMtNGFjNS05NGRiLTQ4ZDQ5M2E2ZDllOCIsInJlc291cmNlR3JvdXBOYW1lIjoiY2hyamlhLXJnIiwiYWNjb3VudE5hbWUiOiJjaHJqaWEtZWFzdHVzMmV1YXAyIiwid29ya3NwYWNlSWQiOiI1OTkzZGVjMS01OTczLTQ4NTktOGJlMS03MDczMzJjNTQ5NWQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in b9133adedd65
 ---> 3e1f75cb9bec
Step 4/5 : RUN mv '/var/azureml-app/tmp2dq72mef.py' /var/azureml-app/main.py
 ---> Running in 92ae17baf213
 ---> 28b53bc2fe7e
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in a34c42c2b58c
 ---> 8f4861264638
Successfully built 8f4861264638
Successfully tagged myservice:latest
Starting Docker container...
Docker container running.
Checking container health...
Local webservice is running at http://localhost:6789


In [8]:
print(service.get_logs())

2021-04-29T13:39:52,836206100+00:00 - gunicorn/run 
2021-04-29T13:39:52,836659900+00:00 - rsyslog/run 
2021-04-29T13:39:52,836681200+00:00 - iot-server/run 
2021-04-29T13:39:52,836978100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## Call into your model

In [10]:
import requests
import json

uri = service.scoring_uri
requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}


Notice the use of the AZUREML_MODEL_DIR environment variable to locate your registered model. Now that you've added some pip packages, you also need to update your inference configuration to add in those additional packages

In [13]:
env = Environment(name='myenv')
python_packages = ['nltk', 'numpy', 'onnxruntime']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inf_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')

## Deploy again and call your service

In [14]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM chrjiaeastusc6ba5291.azurecr.io/azureml/azureml_673d52b5abe224c83574bc767a17fa28
 ---> 0884414da052
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 1ee1a993a131
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjEzZTUwODQ1LTY3YmMtNGFjNS05NGRiLTQ4ZDQ5M2E2ZDllOCIsInJlc291cmNlR3JvdXBOYW1lIjoiY2hyamlhLXJnIiwiYWNjb3VudE5hbWUiOiJjaHJqaWEtZWFzdHVzMmV1YXAyIiwid29ya3NwYWNlSWQiOiI1OTkzZGVjMS01OTczLTQ4NTktOGJlMS03MDczMzJjNTQ5NWQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 760554226dee
 ---> e7dbccdb434f
Step 4/5 : RUN mv '/var/azureml-app/tmpu_afdvkm.py' /var/azureml-app/main.py
 ---> Running in 476b3f45724e
 ---> abc4be1a93d5
Step 5/5 : 

In [15]:
print(service.get_logs())

2021-04-29T14:41:15,891793800+00:00 - iot-server/run 
2021-04-29T14:41:15,892844700+00:00 - rsyslog/run 
2021-04-29T14:41:15,893396800+00:00 - gunicorn/run 
2021-04-29T14:41:15,893587400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

Then ensure you can send a post request to the service:

In [17]:
import requests

uri = service.scoring_uri

headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f04fc5b4650>: Failed to establish a new connection: [Errno 111] Connection refused'))

## Re-deploy to cloud

Once you've confirmed your service works locally and chosen a remote compute target, you are ready to deploy to the cloud.

Change your deploy configuration to correspond to the compute target you've chosen, in this case Azure Container Instances.

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, memory_gb = 1)

Deploy your service again

In [ ]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

## Call your remote webservice

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [ ]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name='myservice')
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
primary_key, _ = service.get_keys()

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response and logs
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

In [ ]:
print(service.get_logs())

## Delete resources

In [ ]:
service.delete()
model.delete()

## Next Steps

Try reading [our documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)